In [1]:
import json 
from transformers import AutoModel, AutoTokenizer, AutoFeatureExtractor, AutoImageProcessor, AutoProcessor, AutoModelForImageClassification
from transformers import AutoModelForDepthEstimation, AutoModelForImageClassification, AutoModelForVideoClassification, AutoModelForMaskedImageModeling, AutoModelForObjectDetection, AutoModelForImageSegmentation, AutoModelForImageToImage, AutoModelForSemanticSegmentation, AutoModelForInstanceSegmentation, AutoModelForUniversalSegmentation, AutoModelForZeroShotImageClassification, AutoModelForZeroShotObjectDetection
from transformers import (
    AutoModelForCausalLM, AutoModelForMaskedLM, AutoModelForSeq2SeqLM, 
    AutoModelForSequenceClassification, AutoModelForMultipleChoice, 
    AutoModelForNextSentencePrediction, AutoModelForTokenClassification, 
    AutoModelForQuestionAnswering, AutoModelForTextEncoding
)
from src.components.LogME import LogME
from src.components.LEEP import LEEP
from src.components.NCE import NCE
import os
import torch 

/home/aksha/Workbench/model_selection/metrics/model-selection-metrics/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-16 09:43:25.438514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 09:43:25.438576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 09:43:25.439407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 09:43:25.445078: I tensorflow/core/platform/cp

In [2]:
import warnings
from PIL import Image
import glob
import pandas as pd
import numpy as np

In [3]:
from tqdm import tqdm
import json

In [4]:
import json 

class Models: 
    def __init__(self): 
        self.tasks_all = [
            "Feature Extraction",
            "Text-to-Image",
            "Image-to-Text",
            "Image-to-Video",
            "Text-to-Video",
            "Visual Question Answering",
            "Document Question Answering",
            "Graph Machine Learning",
            "Text-to-3D",
            "Image-to-3D",
            "Depth Estimation",
            "Image Classification",
            "Object Detection",
            "Image Segmentation",
            "Image-to-Image",
            "Unconditional Image Generation",
            "Video Classification",
            "Zero-Shot Image Classification",
            "Mask Generation",
            "Zero-Shot Object Detection",
            "Text Classification",
            "Token Classification",
            "Table Question Answering",
            "Question Answering",
            "Zero-Shot Classification",
            "Translation",
            "Summarization",
            "Conversational",
            "Text Generation",
            "Text2Text Generation",
            "Fill-Mask",
            "Sentence Similarity",
            "Text-to-Speech",
            "Text-to-Audio",
            "Automatic Speech Recognition",
            "Audio-to-Audio",
            "Audio Classification",
            "Voice Activity Detection",
            "Tabular Classification",
            "Tabular Regression",
            "Reinforcement Learning",
            "Robotics"
        ]
        self.tasks_all = [t.lower().replace(' ', '-') for t in self.tasks_all]
        self.task_models = self.create_task_model_mapping(self.load_json_records()) 
        self.idx = 0 

    def __call__(self, task, n):
        if task in self.task_models:
            self.idx = n
            return self.task_models[task][self.idx:self.idx+n]
        else:
            return []
            
    def create_task_model_mapping(self, json_records, truncate=False):
        task_model_mapping = {}
        checks = {} 
        for task_one in self.tasks_all: 
            task_model_mapping[task_one] = [] 
            checks[task_one] = []
        

        for record in json_records:
            tasks = record[0].split(',')
            model_identifier = record[1]  
            base_model_identifier = record[2]
            dataset = record[4]
            metric = record[5]

            for task in tasks:
                if task in task_model_mapping:
                    if model_identifier not in task_model_mapping[task]:
                        if (model_identifier, base_model_identifier) not in checks[task]:
                            checks[task].append((model_identifier, base_model_identifier))
                            task_model_mapping[task].append([model_identifier, base_model_identifier, dataset, metric])

        if truncate: 
            task_models = {} 
            for task, models in task_model_mapping.items():
                if len(models) > 5:
                    task_models[task] = models
            return task_models

        return task_model_mapping
    
    def load_json_records(self):
        with open('mapping.json') as f: 
            records = json.load(f)
        return records

In [14]:
class Benchmarker:
    def __init__(self, models: Models, logme=True, ckpt=False, regression = False, auto_increment_if_failed = False, leep=False, nce=False, test = False):
        self.models = models
        self.checkpoint = ckpt
        self.auto_increment_if_failed = auto_increment_if_failed
        self.test = test
        if self.test: 
            print('----------Testing Benchmarker----------')

        if int(logme) + int(leep) + int(nce) != 1:
            raise ValueError("ERROR: Only one of logme, leep, or nce can be True") 
        
        if logme:
            self.benchmark_model = LogME(regression=regression) 
            self.regression = regression    
            print(f'SUCCESS: LogME initialized with {"" if regression else "no"} regression')
        elif leep: 
            self.benchmark_model = LEEP()
            print('SUCCESS: LEEP initalized')
        else: 
            self.benchmark_model = NCE()
            print('SUCCESS: NCE initalized')
        
        self.extractor_groups = {
            TextFeatureExtractor: [
                'text-classification',
                'token-classification',
                'question-answering',
                'zero-shot-classification',
                'translation',
                'summarization',
                'conversational',
                'text-generation',
                'text2text-generation',
                'fill-mask',
                'sentence-similarity', 
                'feature-extraction'
            ],
            ImageFeatureExtractor: [
                'image-classification',
                'object-detection',
                'image-segmentation', 
                'video-classification'
            ],
            "Multimodal Not Supported": [
                'text-to-image',
                'image-to-text', 
                'table-question-answering'
            ],
            AudioFeatureExtractor: [
                'automatic-speech-recognition',
                'audio-classification'
            ],
        }

    def __call__(self, design_specs):
        task = design_specs['task']
        dataloader = design_specs['dataset']
        if not isinstance(dataloader, DataLoader): 
            raise ValueError('Dataset should be a `DataLoader` class!!') 
        n = design_specs['n']
        return self.benchmark_models(task, dataloader, n)

    def benchmark_models(self, task, dataloader, n=5): 
        if self.checkpoint:
            checkpoint_pth = f'./checkpoints/{task}_{str(dataloader)}' 
            try: 
                os.mkdir(checkpoint_pth)
            except: 
                pass

        extractor_group = [x for x in self.extractor_groups if task in self.extractor_groups[x]][0]
        if type(extractor_group) == str: 
            raise ValueError(f'ERROR: {extractor_group}')
        
        print('TASK: ', task)

        models = self.models(task, n)
        
        print('MODELS: ', [model[0] for model in models])

        if len(models) == 0:
            raise Error('ERROR: No models selected!')
        
        benchmarks = {}
        
        check = False
        for model_name, base_model_identifier, _, _ in models: 
            
            if check and self.auto_increment_if_failed: 
                self.models(task, 1)
            
            check = False
            
            print('----------------------------------------')
            print('Model_name', model_name)
            if self.checkpoint:
                model_checkpoint_pth = checkpoint_pth + f"/{model_name.replace('/', '_')}/"
                try: 
                    os.mkdir(model_checkpoint_pth)
                except: 
                    pass 
            
            use_checkpoint = False 
            if os.path.exists(model_checkpoint_pth+'/config.json'): 
                try: 
                    config = json.load(model_checkpoint_pth+'/config.json')
                    use_checkpoint=True
                except: 
                    use_checkpoint=False

            if not self.test: 
                if not use_checkpoint:
                    extractor = extractor_group(task, model_name)
                    if not extractor.feature_extractor or not extractor.model:
                        print(f'     ERROR: Loading {model_name}, aborting benchmark!')
                        check = True 
                        continue 
                    else: 
                        pass
                else: 
                    pass
            print('     SUCCESS: Extractor Loaded')
            if not self.test: 
                if not use_checkpoint: 
                    features, targets = self.extract_features_and_targets(extractor, dataloader, n)

                    # try: 
                    #     features, targets = self.extract_features_and_targets(extractor, dataloader, n)
                    # except ValueError as e: 
                    #     print('Error with Extracting: ', e)
                    #     check = True 
                    #     continue    
                else: 
                    features, targets = config['features_pth'], config['targets_pth']

            print('     SUCCESS: Feature Extraction Complete.')
            print('     NOTE: Starting fitting the benchmarker.')
            if not self.test: 
                score = self.fit(features, targets)
            else: 
                score = 1
            print(f'     SUCCESS: Benchmarker completed with score: {score}.')
            print('----------------------------------------')

            benchmarks[model_name] = score
            
            if self.checkpoint:
                np.save(model_checkpoint_pth+'/features.npy', features)
                np.save(model_checkpoint_pth+'/targets.npy', targets)
                config = { 
                    'model_name': model_name,
                    'dataset_name': str(dataloader),
                    'score': score,
                    'features_pth': model_checkpoint_pth+'/features.npy',
                    'targets_pth': model_checkpoint_pth+'/targets.npy'
                }
                with open(model_checkpoint_pth+'/config.json', 'w') as fp:
                    json.dump(config, fp)
            del use_checkpoint, features, targets
        return benchmarks
    
    def extract_features_and_targets(self, extractor, dataloader, n):
        print('     NOTE: Extracting Features and Targets')
        dataloader.reset()
        features_list = []
        labels_list = []

        progress_bar = tqdm(total=dataloader.max_idx, desc='       Progress: ')

        batch = dataloader() 
        while batch: 
            features, labels = extractor(batch)
            
            if not len(features) and not len(labels): 
                batch = dataloader()
                continue

            features_list.append(features)
            if type(labels) != np.ndarray: 
                labels = np.array([labels])
            labels_list.append(labels)

            progress_bar.update(1)

            batch = dataloader()

        progress_bar.close()

        if len(features_list) < n//2: 
            raise ValueError('ERROR: Not enough features extracted! (Could be sampling_rate error for AudioModels).')

        f = np.concatenate(features_list, axis=0)
        y = np.concatenate([np.array(self.convert_labels_with_pandas(labels_list))], axis=0)

        f = f.reshape(f.shape[0], -1)

        if not self.regression:
            y = y.astype(int)
        else:
            if len(y.shape) == 1:
                y = y.reshape(-1, 1)

        print('     NOTE: Features Shape: ', f.shape)
        print('     NOTE: Labels Shape: ', y.shape)

        return f, y
    
    def convert_labels_with_pandas(self, labels_list):
        flat_labels = [label for sublist in labels_list for label in sublist]
        labels_series = pd.Series(flat_labels)
        labels_series = labels_series.astype('category').cat.codes
        return labels_series.values
            
    def fit(self, features, labels):
        warnings.filterwarnings("ignore")
        self.benchmark_model.reset()
        return self.benchmark_model.fit(features, labels)
    
    def is_regression(self): 
        if self.test: 
            print('Testing is_regression')
            return False

        if all(isinstance(label, (int, float)) for label in self.labels):
            unique_labels = set(labels)
            if len(unique_labels) > len(labels) * 0.1:  
                return True

            if all(isinstance(label, int) for label in labels):
                return False

        return False

In [15]:
class TextFeatureExtractor:
    def __init__(self, task_name:str, model_name: str):
        self.feature_extractor = self.load_feature_extractor(model_name)
        self.model = self.load_model(task_name, model_name)

    def load_feature_extractor(self, model_name):
        warnings.simplefilter("error")
        classes = [AutoTokenizer, AutoFeatureExtractor] 
        for class_ in classes:
            try:
                return class_.from_pretrained(model_name)
            except Exception as e:
                pass
            except OSError as e: 
                pass 
        return None


    def load_model(self, task_name, model_name):
        classes = {
            "text-classification": [
                AutoModelForSequenceClassification,
            ],
            "token-classification": [
                AutoModelForTokenClassification,
            ],
            "question-answering": [
                AutoModelForQuestionAnswering,
            ],
            "zero-shot-classification": [
                AutoModelForSequenceClassification,
            ],
            "translation": [
                AutoModelForSeq2SeqLM,
            ],
            "summarization": [
                AutoModelForSeq2SeqLM,
            ],
            "conversational": [
                AutoModelForCausalLM,
            ],
            "text-generation": [
                AutoModelForCausalLM,
            ],
            "text2text-generation": [
                AutoModelForSeq2SeqLM,
            ],
            "fill-mask": [
                AutoModelForMaskedLM,
            ],
            "sentence-similarity": [
                AutoModelForSequenceClassification,
            ],
            "feature-extraction": [
                AutoModelForTextEncoding,
            ]
            }
        warnings.simplefilter("error")

        for task in classes: 
            if task in task_name: 
                for model_class in classes[task]: 
                    try: 
                        return model_class.from_pretrained(model_name)
                    except: 
                        pass
        return None
        
    def __call__(self, text):
        # Text is a tuple 
        if type(text) != tuple: 
            raise ValueError("Input is not a tuple")
        b = self.feature_extractor(text[0], return_tensors="pt", padding='max_length', truncation=True)
        with torch.no_grad(): 
            c = self.model(**b)
        return c.logits.numpy(), text[1]

class ImageFeatureExtractor:
    def __init__(self, task_name:str, model_name: str):
        self.feature_extractor = self.load_feature_extractor(model_name)
        self.model = self.load_model(task_name, model_name)
    
    def load_feature_extractor(self, model_name): 
        warnings.simplefilter("error")
        try: 
            return AutoFeatureExtractor.from_pretrained(model_name)
        except: 
            print('     NOTE: Switching AutoLoader')
            try: 
                return AutoImageProcessor.from_pretrained(model_name)
            except: 
                return None
            
    def load_model(self, task_name, model_name): 
        classes = {'depth-estimation': [AutoModelForDepthEstimation], 
            'image-classification': [AutoModelForImageClassification], 
            'video-classification': [AutoModelForVideoClassification], 
            'object-detection': [AutoModelForObjectDetection], 
            'image-segmentation': [AutoModelForImageSegmentation, AutoModelForUniversalSegmentation, AutoModelForSemanticSegmentation, AutoModelForInstanceSegmentation], 
            'zero-shot-image-classification': [AutoModelForZeroShotImageClassification]}
        warnings.simplefilter("error")

        for task in classes: 
            if task in task_name: 
                for model_class in classes[task]: 
                    try: 
                        return model_class.from_pretrained(model_name)
                    except: 
                        pass
        return None
                

    def __call__(self, image):
        # Image is a tuple here
        if type(image) != tuple: 
            print(image)
            raise ValueError("Input is not a tuple")
        b = self.feature_extractor(images=image[0], return_tensors="pt")
        with torch.no_grad(): 
            c = self.model(**b)
        return c.logits.numpy(), image[1]

class DataLoader: 
    def __init__(self): 
        pass 

class ImageDataLoader(DataLoader): 
    def __init__(self, inputs: list[list[str, str]], name, shuffle=False, n=None):
        self.name=name
        self.img_paths = [x[0] for x in inputs] 
        self.targets = [x[1] for x in inputs]
        self.idx = 0
        self.max_idx = n
        self.len = len(self.targets)
        self.shuffle = shuffle  
        self.dataframe = self.convert_to_pd()

    def __str__(self): 
        return self.name
        
    def __len__(self):
        return self.len  
    
    def convert_to_pd(self): 
        df = pd.DataFrame({'img_path': self.img_paths, 'target': self.targets})
        if self.shuffle: 
            df = df.sample(frac=1).reset_index(drop=True)
        return df
    
    def __call__(self): 
        self.idx += 1
        if (self.idx >= self.len) or (self.max_idx and self.idx > self.max_idx):
            print('     Dataloader exhausted!') 
            return None
        row = self.dataframe.iloc[self.idx]
        image = None
        with Image.open(row['img_path']) as f: 
            image = np.array(f)
        return (image, row['target'])
    
    def reset(self): 
        self.idx = 0   

class TextDataLoader(DataLoader): 
    def __init__(self, inputs: list[list[str, str]], name, shuffle=False, n=None):
        self.name=name
        self.texts = [x[0] for x in inputs] 
        self.targets = [x[1] for x in inputs]
        self.idx = 0
        self.max_idx = n
        self.len = len(self.targets)
        self.shuffle = shuffle  
        self.dataframe = self.convert_to_pd()
    
    def __str__(self): 
        return self.name
        
    def __len__(self):
        return self.len  
    
    def convert_to_pd(self): 
        df = pd.DataFrame({'input': self.texts, 'target': self.targets})
        if self.shuffle: 
            df = df.sample(frac=1).reset_index(drop=True)
        return df
    
    def __call__(self): 
        self.idx += 1
        if (self.idx >= self.len) or (self.max_idx and self.idx > self.max_idx):
            print('     Dataloader exhausted!') 
            return None
        row = self.dataframe.iloc[self.idx]
        return (row['input'], row['target'])
    
    def reset(self): 
        self.idx = 0   

class AudioDataLoader(DataLoader):
    def __init__(self, inputs: list[list[str, str]], name, type: str, normalized:bool, shuffle=False, n=None):
        self.name=name
        self.audio_paths = [x[0] for x in inputs] 
        self.targets = [x[1] for x in inputs]
        self.type=type
        self.normalized=normalized
        self.idx = 0
        self.max_idx = n
        self.len = len(self.targets)
        self.shuffle = shuffle  
        self.dataframe = self.convert_to_pd()

    def __str__(self): 
        return self.name
        
    def __len__(self):
        return self.len  
    
    def convert_to_pd(self): 
        df = pd.DataFrame({'audio_path': self.audio_paths, 'target': self.targets})
        if self.shuffle: 
            df = df.sample(frac=1).reset_index(drop=True)
        return df
    
    def __call__(self): 
        self.idx += 1
        if (self.idx >= self.len) or (self.max_idx and self.idx > self.max_idx):
            print('     Dataloader exhausted!') 
            return None
        row = self.dataframe.iloc[self.idx]
        return ((self.read(row['audio_path'], type=self.type, normalized=self.normalized)), row['target'])
    
    def reset(self): 
        self.idx = 0
    
    def read(self, filename, type='wav', normalized=False):
      with open(filename, 'rb') as file1:
          a = pydub.AudioSegment.from_file(file1, format=type)
          y = np.array(a.get_array_of_samples())
          if a.channels == 2:
              y = y.reshape((-1, 2))
          if normalized:
              return a.frame_rate, np.float32(y) / 2**15
          else:
              return a.frame_rate, np.float32(y)

In [16]:
from transformers import AutoModelForAudioClassification

class AudioFeatureExtractor: 
  def __init__(self, task_name:str, model_name: str):
    self.feature_extractor = self.load_feature_extractor(model_name)
    self.model = self.load_model(task_name, model_name)
  
  def load_feature_extractor(self, model_name): 
    return AutoFeatureExtractor.from_pretrained(model_name)
  
  def load_model(self, task_name, model_name):
    warnings.simplefilter("error")
    classes = {'audio-classification': [AutoModelForAudioClassification, AutoModel]}
    for task in classes: 
        if task in task_name: 
            for model_class in classes[task]: 
                try: 
                    return model_class.from_pretrained(model_name)
                except: 
                    pass
    return None

  def __call__(self, audio):
    # Image is a tuple here
    if type(audio) != tuple: 
        print(audio)
        raise ValueError("Input is not a tuple") 
    try: 
      b = self.feature_extractor(audio[0][1], sampling_rate=audio[0][0], return_tensors="pt")
    except ValueError as e: 
      # Sampling Rate issue
      return None, None

    with torch.no_grad(): 
        c = self.model(**b)
    return c.logits.numpy(), audio[1]


In [17]:
def load_aircraft_targets(): 
  with open("./assets/aircraft/images_family_train.txt") as file: 
    file_list = file.read().splitlines() 
  return [[f'./assets/aircraft/images/{file.split()[0]}.jpg', "-".join(file.split()[1:])] for file in file_list]

def load_tweet_classifier(): 
  df = pd.read_csv('./assets/tweet/Corona_NLP_train.csv', encoding='latin-1')
  df = df[['OriginalTweet', 'Sentiment']]
  return df.astype(str).values.tolist()

def load_esc50_classifier(): 
  df = pd.read_csv('./assets/esc_50/esc50.csv', encoding='latin-1')
  filenames, targets = df['filename'].tolist(), df['target'].tolist()
  return [[f'./assets/esc_50/audio/{filename}', target] for filename, target in zip(filenames, targets)]
  
  
dataloader = ImageDataLoader(load_aircraft_targets(), name='airplane', shuffle=True, n=1000)
# dataloader = TextDataLoader(load_tweet_classifier(), name='tweets', shuffle=True, n=1000)
# dataloader = AudioDataLoader(load_esc50_classifier(), name='esc_50', type='wav', normalized=False, shuffle=True, n=1000)

In [18]:
models = Models()

In [19]:
config = { 
    'task': 'image-classification',
    'dataset': dataloader,  
    'n': 3
}

In [20]:
benchmarker = Benchmarker(models, ckpt=True, logme=True, regression=True, auto_increment_if_failed=False)

SUCCESS: LogME initialized with  regression


In [21]:
benchmarker(config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


TASK:  image-classification
MODELS:  ['Intel/vit-base-patch16-224-int8-static', 'MazenAmria/swin-tiny-finetuned-cifar100', 'OpenGVLab/internimage_xl_1k_384']
----------------------------------------
Model_name Intel/vit-base-patch16-224-int8-static
     NOTE: Switching AutoLoader
     ERROR: Loading Intel/vit-base-patch16-224-int8-static, aborting benchmark!
----------------------------------------
Model_name MazenAmria/swin-tiny-finetuned-cifar100


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


     NOTE: Switching AutoLoader
     SUCCESS: Extractor Loaded
     NOTE: Extracting Features and Targets


       Progress:   0%|          | 0/1000 [01:41<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import pydub 
import numpy as np

def read(filename, type='wav', normalized=False):
    """MP3 to numpy array"""
    with open(filename, 'rb') as file1:
        a = pydub.AudioSegment.from_file(file1, format=type)
        y = np.array(a.get_array_of_samples())
        if a.channels == 2:
            y = y.reshape((-1, 2))
        if normalized:
            return a.frame_rate, np.float32(y) / 2**15
        else:
            return a.frame_rate, np.float32(y)



In [ ]:
read('/home/aksha/Workbench/model_selection/metrics/model-selection-metrics/assets/esc_50/audio/1-137-A-32.wav')

(44100, array([292., 293., 290., ..., 112., 126., 126.], dtype=float32))

In [ ]:
import numpy as np

In [ ]:
features = np.load('/home/aksha/Workbench/model_selection/metrics/model-selection-metrics/checkpoints/text-classification_tweets/ASCCCCCCCC_distilbert-base-chinese-amazon_zh_20000/features.npy')
targets = np.load('/home/aksha/Workbench/model_selection/metrics/model-selection-metrics/checkpoints/text-classification_tweets/ASCCCCCCCC_distilbert-base-chinese-amazon_zh_20000/targets.npy')

In [ ]:
targets.shape

(1000, 1)